In [1]:
import csv, sqlite3
import pandas as pd
import prettytable

prettytable.DEFAULT = 'DEFAULT'

In [2]:
# Load sql magic module
%load_ext sql
%sql sqlite:///FinalDB.db

Use Pandas to load the data to dataframes and use these dataframes to load data on to the database FinalDB.db as required tables.

In [3]:
con = sqlite3.connect("FinalDB.db")
cur = con.cursor()

df = pd.read_csv("./ChicagoCensusData.csv")
df1 = pd.read_csv("./ChicagoPublicSchools.csv")
df2 = pd.read_csv("./ChicagoCrimeData.csv")

Establish a connection between SQL magic module and the database FinalDB.db

In [4]:
df.to_sql("Chicago_Census_data", con, if_exists='replace', index=False,method='multi')
df1.to_sql("Chicago_publicschool_data", con, if_exists='replace', index=False,method='multi', chunksize=200)
df2.to_sql("Chicago_Crime_data", con, if_exists='replace', index=False,method='multi')

533

Question 1. Find the total number of crimes recorded in the CRIME table.

In [5]:
%sql SELECT COUNT(*) FROM Chicago_Crime_data

 * sqlite:///FinalDB.db
Done.


COUNT(*)
533


Question 2. List community area names and numbers with per capita income greater than 11000.

In [6]:
%sql SELECT COMMUNITY_AREA_NAME, COMMUNITY_AREA_NUMBER FROM Chicago_Census_data WHERE PER_CAPITA_INCOME > 11000

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
Rogers Park,1.0
West Ridge,2.0
Uptown,3.0
Lincoln Square,4.0
North Center,5.0
Lake View,6.0
Lincoln Park,7.0
Near North Side,8.0
Edison Park,9.0
Norwood Park,10.0


Question 3. List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [7]:
%sql SELECT CASE_NUMBER FROM Chicago_Crime_Data WHERE DESCRIPTION LIKE "%minor%"

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER
HL266884
HK238408


Question 4. List all kidnapping crimes involving a child?

In [8]:
%sql SELECT * FROM Chicago_Crime_data WHERE PRIMARY_TYPE = "KIDNAPPING" AND DESCRIPTION LIKE "%CHILD%"

 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


Question 5. List the kind of crimes that were recorded at schools. (No repetitions)

In [9]:
%sql SELECT DISTINCT PRIMARY_TYPE FROM Chicago_Crime_Data WHERE LOCATION_DESCRIPTION LIKE "%SCHOOL%"

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


Question 6. List the type of schools along with the average safety score for each type.

In [10]:
%sql SELECT "Elementary, Middle, or High School" as School_type, AVG(SAFETY_SCORE) \
FROM Chicago_publicschool_data GROUP BY School_type

 * sqlite:///FinalDB.db
Done.


School_type,AVG(SAFETY_SCORE)
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


Question 7. List 5 community areas with highest % of households below poverty line

In [11]:
%sql SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM Chicago_Census_data ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


Question 8. Which community area is most crime prone? Display the coumminty area number only.

In [12]:
%sql SELECT COMMUNITY_AREA_NUMBER, COUNT(*) AS CRIME_COUNT FROM Chicago_Crime_data \
GROUP BY COMMUNITY_AREA_NUMBER ORDER BY CRIME_COUNT DESC LIMIT 1

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,CRIME_COUNT
25.0,43


Question 9. Use a sub-query to find the name of the community area with highest hardship index.

In [13]:
%sql SELECT COMMUNITY_AREA_NAME, HARDSHIP_INDEX FROM Chicago_Census_data \
WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX) FROM Chicago_Census_data)

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,HARDSHIP_INDEX
Riverdale,98.0


Question 10. Use a sub-query to determine the Community Area Name with most number of crimes?

In [14]:
%sql SELECT COMMUNITY_AREA_NAME, Crime_count FROM (SELECT s.COMMUNITY_AREA_NAME, count(*) AS Crime_count\
    FROM Chicago_Crime_data AS c JOIN Chicago_Census_data AS s \
    ON c.COMMUNITY_AREA_NUMBER = s.COMMUNITY_AREA_NUMBER \
    GROUP BY COMMUNITY_AREA_NAME) AS SUMMARY \
    WHERE SUMMARY.Crime_count = (SELECT MAX(Crime_count) FROM (SELECT COUNT(*) AS crime_count \
    FROM Chicago_Crime_data AS c JOIN Chicago_Census_data AS s ON c.COMMUNITY_AREA_NUMBER = s.COMMUNITY_AREA_NUMBER \
    GROUP BY s.COMMUNITY_AREA_NAME) AS inner_summary)

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,Crime_count
Austin,43
